In [1]:
# Import necessary libraries
import os
import sys
import requests
import time
import logging
import argparse
from typing import Optional

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Create directories
os.makedirs('utils', exist_ok=True)
os.makedirs('results/part_2', exist_ok=True)

In [2]:
API_URL = "https://api-inference.huggingface.co/models/google/flan-t5-base"
headers = {"Authorization": "Bearer"}  # Replace with your actual API key
def query(payload):
    """
    Send a query to the Hugging Face API

    Args:
        payload: Dictionary containing the query parameters

    Returns:
        The API response as JSON
    """
    response = requests.post(API_URL, headers=headers, json=payload)

    # Print raw response content for debugging
    print("Status Code:", response.status_code)
    print("Raw Response:", response.text)

    try:
        return response.json()
    except requests.exceptions.JSONDecodeError:
        return {"error": "Invalid JSON response", "raw_response": response.text}

# Test the query function
test_payload = {"inputs": "What are the symptoms of diabetes?"}
response = query(test_payload)
print(response)

Status Code: 404
Raw Response: Not Found
{'error': 'Invalid JSON response', 'raw_response': 'Not Found'}


In [33]:
# utils/one_off_chat.py

import requests
import argparse
import os

def get_response(prompt, model_name="google/flan-t5-base", api_key=None):
    """
    Get a response from the model

    Args:
        prompt: The prompt to send to the model
        model_name: Name of the model to use
        api_key: API key for authentication (optional for some models)

    Returns:
        The model's response
    """
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {}
    if api_key:
        headers["Authorization"] = f"Bearer {api_key}"

    payload = {"inputs": prompt}
    try:
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()
        data = response.json()
        if isinstance(data, list) and "generated_text" in data[0]:
            return data[0]["generated_text"]
        elif isinstance(data, list) and "summary_text" in data[0]:  # Some models use this
            return data[0]["summary_text"]
        elif isinstance(data, list):
            return str(data[0])
        else:
            return str(data)
    except requests.exceptions.RequestException as e:
        return f"[ERROR] API request failed: {e}"
    except Exception as e:
        return f"[ERROR] Unexpected error: {e}"

def run_chat(model_name, api_key):
    """Run an interactive chat session"""
    print(f"Welcome to the Simple LLM Chat with {model_name}! Type 'exit' to quit.")

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break

        response = get_response(user_input, model_name=model_name, api_key=api_key)
        print(f"Bot: {response}")

def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM")
    parser.add_argument("--model", type=str, default="google/flan-t5-base", help="Hugging Face model name")
    parser.add_argument("--api-key", type=str, default=os.getenv("HF_API_KEY"), help="Hugging Face API key")

    args = parser.parse_args()

    run_chat(model_name=args.model, api_key=args.api_key)

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] [--model MODEL] [--api-key API_KEY]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\ericy\AppData\Roaming\jupyter\runtime\kernel-1da419c8-d3b4-4497-856c-3d86707d96d6.json


SystemExit: 2

C:\Users\ericy\PycharmProjects\DATASCI 223\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3675: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# utils/conversation.py

import requests
import argparse
import os

def get_response(prompt, history=None, model_name="google/flan-t5-base", api_key=None, history_length=3):
    """
    Get a response from the model using conversation history
    
    Args:
        prompt: The current user prompt
        history: List of previous (prompt, response) tuples
        model_name: Name of the model to use
        api_key: API key for authentication
        history_length: Number of previous exchanges to include in context
        
    Returns:
        The model's response
    """
    # TODO: Implement the contextual response function
    # Initialize history if None
    if history is None:
        history = []
        
    # TODO: Format a prompt that includes previous exchanges
    # Get a response from the API
    # Return the response
    pass

def run_chat():
    """Run an interactive chat session with context"""
    print("Welcome to the Contextual LLM Chat! Type 'exit' to quit.")
    
    # Initialize conversation history
    history = []
    
    while True:
        user_input = input("\nYou: ")
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
            
        # TODO: Get response using conversation history
        # Update history
        # Print the response
        
def main():
    parser = argparse.ArgumentParser(description="Chat with an LLM using conversation history")
    # TODO: Add arguments to the parser
    
    args = parser.parse_args()
    
    # TODO: Run the chat function with parsed arguments
    
if __name__ == "__main__":
    main()

In [ ]:
import os
import csv
from pathlib import Path

# Import our chat modules - since we're in the same directory
from one_off_chat import get_response as get_one_off_response
# Optionally import the conversation module if testing that too
# from conversation import get_response as get_contextual_response

def test_chat(questions, model_name="google/flan-t5-base", api_key=None):
    """
    Test the chat function with a list of questions
    
    Args:
        questions: A list of questions to test
        model_name: Name of the model to use
        api_key: API key for authentication
        
    Returns:
        A dictionary mapping questions to responses
    """
    results = {}
    
    for question in questions:
        print(f"Testing question: {question}")
        # Get response using the one-off chat function
        response = get_one_off_response(question, model_name, api_key)
        results[question] = response
        
    return results

# List of healthcare questions to test
test_questions = [
    "What are the symptoms of gout?",
    "How is gout diagnosed?",
    "What treatments are available for gout?",
    "What lifestyle changes can help manage gout?",
    "What foods should be avoided with gout?"
]

def save_results(results, output_file="results/part_2/example.txt"):
    """
    Save the test results to a file
    
    Args:
        results: Dictionary mapping questions to responses
        output_file: Path to the output file
    """
    with open(output_file, 'w') as f:
        # Write header
        f.write("# LLM Chat Tool Test Results\n\n")
        
        # Write usage examples
        f.write("## Usage Examples\n\n")
        f.write("```bash\n")
        f.write("# Run the one-off chat\n")
        f.write("python utils/one_off_chat.py\n\n")
        f.write("# Run the contextual chat\n")
        f.write("python utils/conversation.py\n")
        f.write("```\n\n")
        
        # Write test results
        f.write("## Test Results\n\n")
        f.write("```csv\n")
        f.write("question,response\n")
        
        for question, response in results.items():
            # Format the question and response for CSV
            q = question.replace(',', '').replace('\n', ' ')
            r = response.replace(',', '').replace('\n', ' ')
            f.write(f"{q},{r}\n")
            
        f.write("```\n")

# Run the test and save results
if __name__ == "__main__":
    results = test_chat(test_questions)
    save_results(results)
    print("Test results saved to results/part_2/example.txt")